In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error, r2_score
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
train = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_features.csv')
sample = pd.read_csv('sample_submission.csv')

## Looking at Datasets

### Train Dataset

In [5]:
# sns.pairplot(train)

In [6]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


## Converting `date_recorded` column to datetime

In [10]:
train['date_recorded'] = pd.to_datetime(train['date_recorded'], errors='ignore')

In [16]:
train.date_recorded.head()

0   2011-03-14
1   2013-03-06
2   2013-02-25
3   2013-01-28
4   2011-07-13
Name: date_recorded, dtype: datetime64[ns]

In [17]:
# Earliest recorded date
train.date_recorded.min()

Timestamp('2002-10-14 00:00:00')

In [18]:
# Latest recorded date
train.date_recorded.max()

Timestamp('2013-12-03 00:00:00')

In [7]:
# train dataset number of rows and features
print(f'Train dataset shape:\n{train.shape}')

Train dataset shape:
(59400, 40)


In [ ]:
train.describe()

In [ ]:
train.id.value_counts()

In [ ]:
train.num_private.value_counts()

In [ ]:
train.construction_year.value_counts()

In [ ]:
train.scheme_management.value_counts()

In [ ]:
train.population.value_counts()

In [ ]:
train.isna().sum()

In [237]:
train.water_quality.value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [238]:
train.quality_group.value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

In [239]:
train.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

In [240]:
train.quantity.value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [ ]:
# Number of unique values per feature
train_columns = train.columns

for col in train_columns:
    print(col, len(train[col].unique()))

#### 1. Dropping Columns that Seem Useless

In [ ]:
train1 = train.drop(['construction_year', 'funder', 'installer', 'scheme_name', 'num_private',
                     'date_recorded', 'wpt_name', 'lga', 'ward', 'recorded_by',
                     'latitude', 'longitude', 'subvillage', 'id'], axis=1)

#### 2. Filling Certain Features' NAs with Mode

In [ ]:
# Filling NAs with mode for each respective column
fill_columns = ['public_meeting', 'permit', 'scheme_management']

for col in fill_columns:
    train1[col] = train1[col].fillna(train1[col].mode()[0])

In [ ]:
train1.isna().sum()

#### 3. Creating Dummy Columns

In [ ]:
# Getting dummies for categorical features
train1 = pd.get_dummies(train1, columns=['permit', 'public_meeting', 'scheme_management', 'management', 'management_group', 
                                  'payment', 'payment_type', 'water_quality', 'quality_group','quantity', 
                                  'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 
                                  'waterpoint_type_group', 'extraction_type', 'extraction_type_group',
                                  'extraction_type_class', 'basin', 'region', 'district_code'], drop_first=True)
train1.head()

In [ ]:
train1.shape

In [ ]:
train1.columns

In [ ]:
train1 = train1.drop(['scheme_management_None', 'extraction_type_other - mkulima/shinyanga'],
                    axis=1)

In [ ]:
train1.shape

### Train Lables (Target)

In [ ]:
train_labels.head()

In [ ]:
# target categories (labels)
train_labels.status_group.unique()

In [ ]:
# Here's a psudo Majority Class baseline for train data
train_labels.status_group.value_counts(normalize=True)

### Test Dataset

In [ ]:
test.head()

In [ ]:
# test dataset number of rows and features
print(f'Test dataset shape:\n{test.shape}')

In [ ]:
test.columns

#### 1. Dropping Columns that Seem Useless

In [ ]:
test1 = test.drop(['construction_year', 'funder', 'installer', 'scheme_name', 'num_private',
                     'date_recorded', 'wpt_name', 'lga', 'ward', 'recorded_by',
                     'latitude', 'longitude', 'subvillage', 'id'], axis=1)

In [ ]:
test1.shape

#### 2. Filling Certain Features' NAs with Mode

In [ ]:
test1.isna().sum()

In [ ]:
# Filling NAs with mode for each respective column
fill_columns = ['public_meeting', 'permit', 'scheme_management']

for col in fill_columns:
    test1[col] = test1[col].fillna(test1[col].mode()[0])

In [ ]:
test1.isna().sum()

#### 3. Creating Dummy Columns

In [ ]:
# Getting dummies for categorical features
test1 = pd.get_dummies(test1, columns=['permit', 'public_meeting', 'scheme_management', 'management', 'management_group', 
                                  'payment', 'payment_type', 'water_quality', 'quality_group','quantity', 
                                  'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 
                                  'waterpoint_type_group', 'extraction_type', 'extraction_type_group',
                                  'extraction_type_class', 'basin', 'region', 'district_code'], drop_first=True)
test1.head()

In [ ]:
test1.shape

In [ ]:
for col in train1.columns:
    if col not in test1.columns:
        print(col)

In [ ]:
train1.columns == test1.columns

## Majority Class Baseline

In [ ]:
# setting a y_pred array
# same length as test set & filled with train_label mode
y_pred = np.full(test.shape[0],
                train_labels.status_group.mode()[0])
len(y_pred)

In [ ]:
len(test.id.values)

In [ ]:
majority_baseline_dict = {'id': test.id.values, 
                              'status_group': y_pred}
majority_baseline = pd.DataFrame(data=majority_baseline_dict).set_index('id')

In [ ]:
majority_baseline.head()

In [ ]:
majority_baseline.to_csv('majority_baseline.csv')

## Model Better than Baseline

In [ ]:
def split(x, y):
    # Hold out an "out-of-time" test set, from the last 100 days of data
    
    X_train = x[:-14358]
    X_val = x[-14358:]

    y_train  = y[:-14358]
    y_val  = y[-14358:]
    
    return X_train, X_val, y_train, y_val

In [ ]:
# 14358
X_train, X_val, y_train, y_val = split(train1, train_labels.status_group)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# encoding target features (y_train & y_val)
y_train_encoded = y_train.map({'non functional': 0,
                              'functional needs repair': 1, 'functional': 2})
y_val_encoded= y_val.map({'non functional': 0,
                              'functional needs repair': 1, 'functional': 2})

In [ ]:
y_train_encoded.shape, y_val_encoded.shape

### Defining Transformations & Pipeline

In [ ]:
pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression(multi_class='multinomial'))

In [ ]:
param_grid = {
    'logisticregression__solver': ['lbfgs']
}

In [ ]:
grid = GridSearchCV(pipe, param_grid=param_grid,
                   cv=2)

### Fitting Better Model

In [ ]:
grid.fit(X_train, y_train_encoded)

In [ ]:
grid.best_score_

In [ ]:
grid.best_score_

In [ ]:
grid.score(X_val, y_val_encoded)

In [ ]:
y_pred_grid = grid.predict(test1)
y_pred_grid

### Saving as CSV

In [ ]:
better_model_dict = {'id': test.id.values, 
                              'status_group': y_pred_grid}
better_model = pd.DataFrame(data=better_model_dict).set_index('id')

In [ ]:
better_model.status_group = better_model.status_group.map({0:'non functional', 
                     1:'functional needs repair', 2:'functional'})

In [ ]:
better_model.head()

In [ ]:
better_model.shape

In [ ]:
better_model.to_csv('better_model.csv')

## Model 0

In [ ]:
pipe0 = make_pipeline(
    StandardScaler(),
    SVC())

In [ ]:
param_grid0 = {
    'svc__solver': ['lbfgs']
}

## First Model

In [ ]:
pipe.fit(X_train, y_train_encoded)
y_pred = pipe.predict(X_val)
accuracy_score(y_val_encoded,y_pred)

In [ ]:
y_pred = pipe.predict(X_val)
accuracy_score(y_val_encoded,y_pred)

In [ ]:
y_pred

In [ ]:
first_model_dict = {'id': test.id.values, 
                              'status_group': y_pred}
first_model = pd.DataFrame(data=first_model_dict).set_index('id')

In [ ]:
first_model.status_group = first_model.status_group.map({0:'non functional', 
                     1:'functional needs repair', 2:'functional'})

In [ ]:
first_model.head()

In [ ]:
first_model.status_group.value_counts()

In [ ]:
first_model.shape

In [ ]:
first_model.to_csv('first_model.csv')

## A Tweak in the Last Model

In [ ]:
pipe1 = make_pipeline(
    StandardScaler(),
    LogisticRegression(solver='sag', multi_class='multinomial')) # using different solver - does not help

In [ ]:
pipe1.fit(X_train, y_train_encoded)
y_pred = pipe1.predict(X_val)
accuracy_score(y_val_encoded,y_pred)

## Another Model

In [ ]:
# Try 3-way-holdout method (train, validation, and test) 
# and try model on all 3 sets
#

<img src="https://sebastianraschka.com/images/blog/2018/model-evaluation-selection-part4/model-eval-conclusions.jpg" width="600">

In [ ]:
pipe_ridge = make_pipeline(
    StandardScaler(),
    Ridge())

In [ ]:
pipe_ridge.fit(X_train, y_train_encoded)
y_pred_ridge = pipe_ridge.predict(X_val)
r2_score(y_val_encoded,y_pred_ridge)

In [ ]:
param_grid = {
    'ridge__alpha': [0.0001, 0.001, 0.1, 1.0, 10.] # values of alpha kept to 3 to run faster
}

gs = GridSearchCV(pipe_ridge, param_grid=param_grid, cv=3, 
                  scoring='neg_mean_absolute_error', 
                  verbose=1)

gs.fit(X_train, y_train_encoded)
validation_score = gs.best_score_
-validation_score

In [ ]:
gs.best_estimator_

In [ ]:
y_pred = gs.predict(X_val) 

validation_score = mean_absolute_error(y_val_encoded, y_pred)
print('Validation Score:', validation_score)

In [ ]:
# Here's the predicted y - But how do I interpret this??
# Using LogisticRegression at least values remain
y_pred

## Going Beyond 60%